In [2]:
import urllib.request
import json
import glob
import pandas as pd
import numpy as np
import datetime

Get data from sensors

In [3]:
# this cell gets data
URL = "http://165.227.244.213:8881/luftdatenGet/22FQ8dJEApww33p31935/9d93d9d8cv7js9sj4765s120sllkudp389cm/" 
response = urllib.request.urlopen(URL)
data = json.loads(response.read())
print(data)


[{'_id': '5cfaeac7ba5da8385e4620d9', 'esp8266id': '3652817', 'software_version': 'NRZ-2018-123B', 'sensordatavalues': [{'value_type': 'SDS_P1', 'value': '3.12'}, {'value_type': 'SDS_P2', 'value': '0.60'}, {'value_type': 'samples', 'value': '1721462'}, {'value_type': 'min_micro', 'value': '82'}, {'value_type': 'max_micro', 'value': '21103'}, {'value_type': 'signal', 'value': '-70'}]}, {'_id': '5cfaeb5b79660f38824df851', 'esp8266id': '3652817', 'software_version': 'NRZ-2018-123B', 'sensordatavalues': [{'value_type': 'SDS_P1', 'value': '2.83'}, {'value_type': 'SDS_P2', 'value': '0.70'}, {'value_type': 'samples', 'value': '1719772'}, {'value_type': 'min_micro', 'value': '82'}, {'value_type': 'max_micro', 'value': '21121'}, {'value_type': 'signal', 'value': '-70'}]}, {'_id': '5cfaebf479660f38824df852', 'esp8266id': '3652817', 'software_version': 'NRZ-2018-123B', 'sensordatavalues': [{'value_type': 'SDS_P1', 'value': '3.72'}, {'value_type': 'SDS_P2', 'value': '0.70'}, {'value_type': 'samples

read from json data to be read into pd.DataFrame

In [140]:
columns = ['hex_int','date_time','P1','P2']
df_P = pd.DataFrame()


for values in data:
    chip_id = values.get('esp8266id')
    sensor_vals = values.get('sensordatavalues')
    P1 = sensor_vals[0]['value']
    P2 = sensor_vals[1]['value']
    test_id = values.get('_id')
    sub_test_id = test_id[0:8]
    hex_int = int(sub_test_id, 16) 
    
    date_time = datetime.datetime.utcfromtimestamp((hex_int))# * 1000)
    #date_time = datetime.((testid.substring(0, 8), 16) * 1000)
    
    #print('test_id',test_id,'sub_test_id',sub_test_id,'hex_int',hex_int,'date_time',date_time)
    row = pd.Series([hex_int,date_time,P1, P2])   #'chip_id' has been removed 
    df_row = pd.DataFrame(row).transpose()  
    #print(df_row)
    df_P = pd.concat([df_P,df_row],axis=0)
  
  
df_P = pd.DataFrame(data= df_P.values, columns = columns ) 
df_P = df_P.sort_values(by=['hex_int'],ascending=False, inplace=False)
df_P['P1'] = df_P['P1'].astype('float64')
df_P['P2'] = df_P['P2'].astype('float64')
df_P['hex_int'] = df_P['hex_int'].astype('int64')
#print(df_P)
    

Get columns of interest # Not applicable delete cell, job done in previous cell

In [ ]:
#redundant cell

Resample Data to current data point and means for hour, week, month, year # work only done for current and Daily

In [141]:
print('Current values for the sensor')
current_row = df_P.iloc[0,:]
current_P1 = df_P['P1'][0]
current_P2 = df_P['P2'][0]
print('current_row',current_row)
#This is the latest value
P1= df_P['P1'].values
P2= df_P['P2'].values

P1 = pd.DataFrame(data=P1,index=df_P['date_time'])
P2 = pd.DataFrame(data=P2,index=df_P['date_time'])

daily_P1 = P1.resample('D').mean()
daily_P2 = P2.resample('D').mean()
daily_P1 = daily_P1.sort_values(by=['date_time'],ascending=False, inplace=False)
daily_P2 = daily_P2.sort_values(by=['date_time'],ascending=False, inplace=False)
print(daily_P1)

print(daily_P2)

todays_P1 = daily_P1.iloc[0,:]
todays_P2 = daily_P2.iloc[0,:]
print(todays_P1)
print(todays_P2)

Current values for the sensor
current_row hex_int               1560073725
date_time    2019-06-09 09:48:45
P1                          2.08
P2                          0.72
Name: 844, dtype: object
                       0
date_time               
2019-06-09      1.933502
2019-06-08      2.241207
2019-06-07  62735.290000
                   0
date_time           
2019-06-09  0.722405
2019-06-08  0.654121
2019-06-07  3.521071
0    1.933502
Name: 2019-06-09 00:00:00, dtype: float64
0    0.722405
Name: 2019-06-09 00:00:00, dtype: float64


Output values from here to dashboard

In [145]:
output = {}
output.update({'sensor_id':chip_id})
output.update({'current_PM10':current_P1})
output.update({'current_PM2.5': current_P2})
output.update({'daily_PM10':todays_P1[0]})
output.update({'daily_PM2.5':todays_P2[0]})
print(output)

{'sensor_id': '3652817', 'current_PM10': 3.12, 'current_PM2.5': 0.6, 'daily_PM10': 1.9335021097046416, 'daily_PM2.5': 0.7224050632911396}
